In [1]:
import cv2
import numpy as np
import os
import json
from utils import load_sequence, show_tracked, track_to_file, cossim, load_frame_number
from sort.sort import Sort
from track import track_sequence
from db import DatabaseHandler
from feature_extractor import FeatureExtractor
from PIL import Image

In [2]:
seq = 'S01'
dir_path = './AICity/train'
img_path = './AICity/cam_loc'
roi_path = dir_path + '/roi.jpg'

In [3]:
det_name = 'det/det_tracked.txt'
frame_boundaries = load_frame_number(dir_path, seq, det_name)

In [4]:
frame_boundaries

[[1, 1955], [1, 2110], [1, 1996], [1, 2110], [1, 2110]]

In [4]:
# det_name_in = 'det/det_yolo3.txt'
# track_sequence(dir_path, seq, det_name_in)

In [5]:
camdirs = os.listdir(os.path.join(dir_path, seq))
camdirs

['c001', 'c002', 'c003', 'c004', 'c005']

In [6]:
video_captures, img, bboxes = load_sequence(dir_path, img_path, seq, det_name=det_name, with_id=True)

In [22]:
# seq = 'S01'
# camdir = 'c001'
# det_name_in = 'det/det_tracked.txt'

# tracked = get_bboxes(os.path.join(dir_path, seq, camdir, det_name_in), with_id=True)
# show_tracked(video_captures[0], tracked)

In [7]:
dbh = DatabaseHandler('feature_db_2.db')
fe = FeatureExtractor()

c:\Users\mirun\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\mirun\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
def multicamera_tracking(video_captures, bboxes, seq, dbh, fe, camdirs, frame_boundaries, det_name_out='det/det_multi.txt', start_frame=0, end_frame=None):
    """
    Parameters:
    - video_captures (list): List of videos where each video represents one camera in a sequence.
    - bboxes (list): List of dictionaries where each dictionary represents the tracked bounding boxes in one camera.
                    Each dictionary includes keys as frame indices and values as bounding box coordinates and track_id.
                    Example: [{'frame_1': [[id, x1, y1, x2, y2], [id, x1, y1, x2, y2]], 'frame_2': [[id, x1, y1, x2, y2]]}, ...]
    - start_frame (int): The starting frame index for multi-camera tracking. Default is 0.
    - end_frame (int): The ending frame index for multi-camera tracking. If None, it tracks until the last frame.
                       Default is None.

        
    """
    id_mapping = {}
    camdirs = sorted(camdirs)

    def get_frame(cap, frame_id):
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        _, frame = cap.read()

        return frame

    def process_object(frame, bbox):
        x1, y1, x2, y2 = map(int, bbox)
        bbox_img = frame[y1:y2, x1:x2]
        bbox_img = Image.fromarray(bbox_img)

        return fe.extract_features(bbox_img)

    # Iterate over all bboxes to create DB
    for cam in range(len(bboxes)):
        print(f"Cam: {cam} so camdir: {camdirs[cam]}")

        frame_id, stop_frame = frame_boundaries[cam]
        while frame_id < stop_frame:
            print("frame ", frame_id)

            if not frame_id in bboxes[cam]:
                continue 

            frame_bboxes = bboxes[cam][frame_id]
            frame = get_frame(video_captures[cam], frame_id)

            for bbox in frame_bboxes:
                track_id = bbox[0]

                # Do not extract features again if already in DB
                if dbh.is_track_id_exists(track_id, seq):
                    continue

                # Extract feature of object in bbox and save to DB
                feature_vector = process_object(frame, bbox[1:])
                dbh.insert_feature(track_id, frame_id, seq, cam, feature_vector, False)

            frame_id += 1

    print(f"---------------------------------------")

    
    # Iterate over all cameras to reidentify    
    for cam in range(len(bboxes)):
        print(f"Cam: {cam} so camdir: {camdirs[cam]}")

        frame_id, stop_frame = frame_boundaries[cam]
        bboxes_out = {}

        print(f"Frame ID: {frame_id} stop_frame: {stop_frame}")
        while frame_id < stop_frame:
            print("frame ", frame_id)

            if not frame_id in bboxes[cam]:
                continue 
            
            # Get the bounding boxes in given camera and at certain frame
            frame_bboxes = bboxes[cam][frame_id]
            frame = get_frame(video_captures[cam], frame_id)

            for bbox in frame_bboxes:
                track_id = bbox[0]

                # Extract feature of object in bbox
                feature_vector = process_object(frame, bbox[1:])

                # Perform similarity search if new tracking_id (not in DB)
                curr_obj = dbh.get_object(track_id, seq)

                # Check that this object has not be reidentified already
                if curr_obj and not curr_obj[-1]:
                    # TODO: Add offset according to timestamp 
                    # The issue might be that the first time an object is tracked is way before
                    # it appears in another video
                    objects = dbh.get_between_frames_not_in_camera(frame_id - 50, frame_id + 50, camdirs[cam])
                    max_similarity, max_id = 0, -1
                    for obj in objects:
                        obj_fv = np.frombuffer(obj[-1], dtype=np.float32)

                        similarity = cossim(obj_fv, feature_vector)
                        if similarity > max_similarity:
                            max_similarity, max_id = similarity, obj[0]

                    if max_similarity > 0.85:
                        # Mark object as reidentified
                        dbh.update_reid(track_id, seq, True)

                        # Ensure that the objects do not map to each other
                        if max_id in id_mapping and id_mapping[max_id] != track_id:
                            print(f"{track_id} mapped to {max_id} with {max_similarity} confidence")
                            id_mapping[track_id] = max_id


                reid = id_mapping[track_id] if track_id in id_mapping else track_id
                if frame_id not in bboxes_out:
                    bboxes_out[frame_id] = []
                
                bboxes_out[frame_id].append((frame_id, bbox[1], bbox[2], bbox[3], bbox[4], reid))
            frame_id += 1

        camdir = camdirs[cam]
        track_to_file(bboxes_out, os.path.join(dir_path, seq, camdir, det_name_out))
        

In [9]:
multicamera_tracking(video_captures, bboxes, seq, dbh, fe, camdirs, frame_boundaries)

2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048
2048


KeyboardInterrupt: 